In [3]:
! pwd

/home/clark/Documents/GitHub/CNA_tool/raw_data_extraction


In [26]:
import scanpy as sc
import pandas as pd
from scipy.io import mmread
from pathlib import Path
from anndata import AnnData

# Sample folders
base = Path("../data/GSE194214_RAW")
samples = {
    "d1": "GSM5830919_somitoid_d1",
    "d2": "GSM5830920_somitoid_d2",
    "d3": "GSM5830921_somitoid_d3",
    "d5": "GSM5830922_somitoid_d5",
}

adatas = {}


In [27]:
for day, subdir in samples.items():
    path = base / subdir
    X = mmread(path / "matrix.mtx").T.tocsr()
    var = pd.read_csv(path / "features.tsv", header=None, sep="\t")
    obs = pd.read_csv(path / "barcodes.tsv", header=None, sep="\t")

    adata = AnnData(X=X)
    adata.var["gene_ids"] = var[0].values
    adata.var["gene_symbols"] = var[1].values
    # Ensure uniqueness by appending gene_id if needed
    adata.var_names = pd.Index(adata.var["gene_symbols"]).astype(str)
    adata.var_names_make_unique()
    
    adata.obs_names = obs[0].values
    adata.obs["day"] = day

    adatas[day] = adata


In [28]:
adata_combined = adatas["d1"].concatenate(
    adatas["d2"], adatas["d3"], adatas["d5"],
    batch_key="sample_day",
    batch_categories=["d1", "d2", "d3", "d5"]
)


/tmp/ipykernel_672348/469965187.py:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_combined = adatas["d1"].concatenate(


In [29]:
# Rename the 'gene_symbols' column to 'gene_name'
adata_combined.var.rename(columns={'gene_symbols': 'gene_name'}, inplace=True)

# # Unset the index name to avoid conflicts
# adata_combined.var.index.name = None:contentReference[oaicite:13]{index=13}


In [31]:
adata_combined.write("../data/num5_adata.h5ad")

In [33]:
adata_combined.obs

,day,sample_day
AAACGAATCATAGGCT-1-d1,d1,d1
AAACGAATCGCCATAA-1-d1,d1,d1
AAACGCTCACATGACT-1-d1,d1,d1
AAACGCTGTCGAGATG-1-d1,d1,d1
AAACGCTTCGGAGTGA-1-d1,d1,d1
...,...,...
TTTGTTGCACTGATTG-1-d5,d5,d5
TTTGTTGCATCGGATT-1-d5,d5,d5
TTTGTTGTCCAAACCA-1-d5,d5,d5
TTTGTTGTCCAGTACA-1-d5,d5,d5
